In [1]:
from copy import deepcopy
from numpy.linalg import inv, norm
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn import linear_model
import numpy as np
import pandas as pd

In [2]:
diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:, np.newaxis, 2]
diabetes_Y = diabetes.target.reshape((diabetes_X.shape[0], 1))

### Standard regression (No Regularization)

In [3]:
reg = linear_model.LinearRegression()
reg.fit (diabetes_X, diabetes_Y)
reg.intercept_, reg.coef_

(array([152.13348416]), array([[949.43526038]]))

In [4]:
diabetes_X_all = np.concatenate((np.ones((diabetes_X.shape[0], 1)), diabetes_X), axis=1)

In [5]:
#Closed form
betas_cf = inv(diabetes_X_all.T @ diabetes_X_all) @ diabetes_X_all.T @ diabetes_Y
betas_cf

array([[152.13348416],
       [949.43526038]])

In [6]:
#Gradient descent
eta = 0.001
iter_max = 100000
beta = np.random.randn(diabetes_X_all.shape[1],1)

tolerance = 1e-10
for i in range(iter_max):
    beta_old = deepcopy(beta)
    beta += eta*diabetes_X_all.T @ (diabetes_Y - diabetes_X_all @ beta)
    
    if norm(beta-beta_old) < tolerance:
        print("Achieved convergence at iteration ", i)
        break

beta

Achieved convergence at iteration  22962


array([[152.13348416],
       [949.43526028]])

### Standard regression (Regularization)

In [7]:
X, y = make_regression(n_features=2, random_state=0)
lambda_1 = 0.5
lambda_2 = 0.4
alpha = lambda_1 + 2*lambda_2
l1_ratio = lambda_1 / alpha

rege = linear_model.ElasticNet(alpha=alpha,l1_ratio=l1_ratio,random_state=0)#As we shall see,
#algorithms below are better
rege.fit(X, y)
rege.intercept_, rege.coef_

(1.9021228843501663, array([15.60966377, 54.01512998]))

In [8]:
y_hat = rege.predict(X)
rmse = np.sqrt(np.sum((y-y_hat)**2)/len(y))
rmse

44.94897727346183

In [9]:
#Coordinate descent
X_all = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
n, m = X_all.shape
beta = np.random.randn(m,1)
iter_max = 20
tolerance = 1e-10

for t in range(iter_max):
    beta_old = deepcopy(beta)
    for j in range(m):
        rho_j = 0
        z_j = np.dot(X_all[:,j],X_all[:,j])
        for i in range(n):
            rho_j += (y[i] - (X_all[i,:] @ beta_old - X_all[i,j] * beta_old[j]))*X_all[i,j]

        if rho_j < -lambda_1/2:
            beta[j] = (rho_j + lambda_1/2)/(z_j + lambda_2)
        elif rho_j >= -lambda_1/2 and rho_j <= lambda_1/2:
            beta[j] = 0
        else:
            beta[j] = (rho_j - lambda_1/2)/(z_j + lambda_2)
    
    if norm(beta-beta_old) < tolerance:
        print("Achieved convergence at iteration ", t)
        break

beta

Achieved convergence at iteration  14


array([[1.47645526e-02],
       [2.90909233e+01],
       [9.58212956e+01]])

In [10]:
rmse = 0
y_hat = np.zeros(y.shape)
for i in range(n):
    y_hat[i] = X_all[i,:]@beta

rmse = np.sqrt(np.sum((y-y_hat)**2)/len(y))
rmse

0.39790094956906014

In [11]:
#Gradient descent
eta = 0.001
iter_max = 1000
tolerance = 1e-10
beta = np.random.randn(m,1)

for t in range(iter_max):
    beta_old = deepcopy(beta)
    for j in range(m):
        dbeta_j = 0
        
        rho_j = 0
        z_j = np.dot(X_all[:,j],X_all[:,j])
        for i in range(n):
            rho_j += (y[i] - (X_all[i,:] @ beta_old - X_all[i,j] * beta_old[j]))*X_all[i,j]
        if beta[j] < 0:
            dbeta_j = -2*rho_j + 2*beta[j]*z_j - lambda_1 + 2*lambda_2*beta[j]
        elif beta[j] == 0:#a + random*(b-a)
            dbeta_j = (-2*rho_j - lambda_1) + (np.random.rand()*(2*lambda_1))
        else:
            dbeta_j = -2*rho_j + 2*beta[j]*z_j + lambda_1 + 2*lambda_2*beta[j]
        
        beta[j] -= eta*dbeta_j
    
    if norm(beta-beta_old) < tolerance:
        print("Achieved convergence at iteration ", t)
        break

beta

Achieved convergence at iteration  127


array([[1.47645529e-02],
       [2.90909233e+01],
       [9.58212956e+01]])

In [12]:
rmse = 0
y_hat = np.zeros(y.shape)
for i in range(n):
    y_hat[i] = X_all[i,:]@beta

rmse = np.sqrt(np.sum((y-y_hat)**2)/len(y))
rmse

0.3979009496956956